In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 生成示例数据（正弦波 + 噪声）
def generate_data(length=300):
    x = np.linspace(0, 10, length)
    y = np.sin(x) + np.random.normal(0, 0.1, length)
    return y.reshape(-1, 1)

In [ ]:
# 数据预处理
def create_dataset(data, time_step=10):
    X, Y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        Y.append(data[i+time_step, 0])
    return np.array(X), np.array(Y)

In [ ]:
# 数据准备
data = generate_data()
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)


In [ ]:
# 划分训练集和测试集
train_size = int(len(data)*0.8)
test_size = len(data) - train_size
train, test = data[0:train_size], data[train_size:]

In [ ]:
# 创建训练和测试数据集
time_step = 20
X_train, y_train = create_dataset(train, time_step)
X_test, y_test = create_dataset(test, time_step)

In [ ]:
# 调整输入形状为 [样本数, 时间步长, 特征数]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# 创建LSTM模型
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# 训练模型
history = model.fit(X_train, y_train, 
                    epochs=20, 
                    batch_size=16, 
                    validation_split=0.1,
                    verbose=1)

In [ ]:
# 进行预测
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# 反向归一化
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

In [ ]:
# 可视化结果
plt.figure(figsize=(12, 6))

# 原始数据
plt.plot(scaler.inverse_transform(data), label='Original Data')

# 训练预测
train_plot = np.empty_like(data)
train_plot[:, :] = np.nan
train_plot[time_step:len(train_predict)+time_step, :] = train_predict
plt.plot(train_plot, label='Training Predictions')

# 测试预测
test_plot = np.empty_like(data)
test_plot[:, :] = np.nan
test_plot[len(train_predict)+(time_step*2)+1:len(data)-1, :] = test_predict
plt.plot(test_plot, label='Testing Predictions')

plt.title('LSTM Sequence Predictions')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.show()